In [1]:
import pickle

import pandas as pd
import scipy.sparse
import scipy.sparse

from lightfm import LightFM
from lightfm.data import Dataset

In [2]:
df = pd.read_parquet('/pio/scratch/1/recommender_systems/processed/amazon-clothes/5-core/train.parquet')
test_df = pd.read_parquet('/pio/scratch/1/recommender_systems/processed/amazon-clothes/5-core/test.parquet')
categories = pd.read_json('/pio/scratch/1/recommender_systems/interim/Amazon/meta_Clothing_Shoes_and_Jewelry_categories.json', lines=True)

In [3]:
interactions = scipy.sparse.load_npz('/pio/scratch/1/i313924/data/lightfm_data/5_core_interactions.npz')
test_interactions = scipy.sparse.load_npz('/pio/scratch/1/i313924/data/lightfm_data/5_core_test_interactions.npz')
dataset = pd.read_pickle('/pio/scratch/1/i313924/data/lightfm_data/5_core_dataset.pkl')

In [4]:
word2vec_embeddings = pd.read_parquet('/pio/scratch/1/recommender_systems/processed/word2vec/amazon-clothes/5-core/item_item_embeddings.parquet')

In [5]:
item_mapping = dataset.mapping()[2]
word2vec_embeddings = word2vec_embeddings.rename(index=item_mapping)
word2vec_embeddings = word2vec_embeddings.sort_index()

In [6]:
items = df['asin'].drop_duplicates()
item_mapping = dataset.mapping()[2]
items_categories = categories[categories.asin.isin(items)]
women_ids = items_categories[items_categories['category_1'] == 'Women']['asin'].unique()
women_ids = [item_mapping[id] for id in women_ids]
men_ids = items_categories[items_categories['category_1'] == 'Men']['asin'].unique()
men_ids = [item_mapping[id] for id in men_ids]

In [7]:
model = LightFM(no_components=100, learning_schedule='adadelta', loss='warp', women_ids=women_ids, men_ids=men_ids)

my implementation


In [8]:
num_epochs = [5, 25, 50, 250, 500, 1000, 1500]
remaining = [num_epochs[0]] + [num_epochs[i] - num_epochs[i-1] for i in range(1, len(num_epochs))]
for i, epochs in enumerate(num_epochs):
    # model.fit_partial(interactions, verbose=True, epochs=remaining[i], num_threads=8, word2vec_embeddings=word2vec_embeddings.values)
    model.fit_partial(interactions, verbose=True, epochs=remaining[i], num_threads=8)
    pickle.dump(model, open(f'/pio/scratch/1/i313924/data/lightfm_data/model_{epochs}_epochs_regularized_partial.pkl', 'wb'), protocol=4)

Epoch: 100%|██████████| 500/500 [49:23<00:00,  5.93s/it]
